In [84]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
import torch
import copy
import torch.nn as nn
import torch.optim as optim
from utils import data_loader, train, eval, nn_modules
from sklearn.preprocessing import MinMaxScaler
from alpha_vantage.timeseries import TimeSeries

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
API_KEY = 'PZZ6AOYX66Q8H83Q'
ts = TimeSeries(key=API_KEY, output_format='pandas', indexing_type='date')
data, meta_data = ts.get_intraday('FB')
meta_data

{'1. Information': 'Intraday (15min) open, high, low, close prices and volume',
 '2. Symbol': 'FB',
 '3. Last Refreshed': '2019-03-29 16:00:00',
 '4. Interval': '15min',
 '5. Output Size': 'Compact',
 '6. Time Zone': 'US/Eastern'}

In [90]:
batch_size = 1
input_size = 1
hidden_size = 50
num_layers = 2
prediction_window = 3
training_window = 10
dropout=0.2
epochs = 40

# x_train, y_train, x_val, y_val = load_data(fb_df, training_window=training_window, prediction_window=prediction_window)

dfs = data_loader.load_daily_data('./data/FAANG_13_18_CSV.csv', ['FB'])
x_train, y_train, x_val, y_val = data_loader.create_split(dfs[0], training_window=training_window, prediction_window=prediction_window)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dev_cpu = torch.device("cpu")
net = nn_modules.StockPredictor(input_size, hidden_size, num_layers, device, prediction_window, batch_size=batch_size, p=0.2)
train_itr = data_loader.BatchIterator((x_train, y_train), batch_size)
val_itr = data_loader.BatchIterator((x_val, y_val), batch_size)

loss_function = nn.MSELoss(reduction='sum')
optimizer = optim.Adam(net.parameters(), lr=1e-3)

losses, min_val_mse = train.train(net, train_itr, val_itr, loss_function, optimizer, device, epochs=epochs)


Epoch:0, Average loss:0.0017757054883986712
Average val MSE = 0.034703770700486886
Epoch:1, Average loss:0.01124353613704443
Average val MSE = 0.03309191255878641
Epoch:2, Average loss:0.01295701414346695
Average val MSE = 0.03127929906940805
Epoch:3, Average loss:0.013563079759478569
Average val MSE = 0.030955091148929203
Epoch:4, Average loss:0.01386948861181736
Average val MSE = 0.031634004816520636
Epoch:5, Average loss:0.017848920077085495
Average val MSE = 0.031403196121019224
Epoch:6, Average loss:0.015991998836398125
Average val MSE = 0.03004262894654278
Epoch:7, Average loss:0.014698855578899384
Average val MSE = 0.03581397960863868
Epoch:8, Average loss:0.018684986978769302
Average val MSE = 0.02756717085532888
Epoch:9, Average loss:0.02507413551211357
Average val MSE = 0.02375791159562316
Epoch:10, Average loss:0.025342721492052078
Average val MSE = 0.022319061264560507
Epoch:11, Average loss:0.0255658607929945
Average val MSE = 0.03389013468827253
Epoch:12, Average loss:0.0

In [92]:
net = eval.load_saved_model('./models/best_model.pt', net)
eval.evaluate_model(net, val_itr)

# net.load_state_dict(torch.load('./models/best_model.pt'))
# net.eval()
# print('Sucessfully loaded model {}'.format('./models/best_model.pt'))

Sucessfully loaded model ./models/best_model.pt
Average test MSE = 0.012070760901629255
